### VARIABLE NAME AND THEIR DESCRIPTION

In [ ]:
import re
import pandas as pd

In [ ]:
do_file = r'/path/to/.DO/file'
variable_mapping = {}

with open(do_file, 'r') as file:
    lines = file.readlines()

# Parse label values
for line in lines:
    if line.startswith('label values'):
        parts = line.split()
        subvar = parts[2]
        var = parts[3]
        if var not in variable_mapping:
            variable_mapping[var] = []
        variable_mapping[var].append(subvar)

In [ ]:
variable_info = {}
# Parse label variable
for line in lines:
    if line.startswith('label variable'):
        parts = line.split()
        subvar = parts[2]
        var_name = subvar.upper().split('_')[0]
        description = ' '.join(parts[3:]).strip('"')
        if var_name in variable_mapping.keys():
            # Find the corresponding variable for the subvariable
            for var, subvars in variable_mapping.items():
                if subvar in subvars:
                    if var not in variable_info:
                        variable_info[var] = {'description': description, 'subnames': subvars, 'values': {}}
                    break
        else:
            if var_name not in variable_info:
                variable_info[var_name] = {'description': description, 'subnames': [subvar], 'values': {}}  

In [ ]:
current_var = None
for line in lines:
    if line.startswith('label define'):
        parts = line.split()
        current_var = parts[2]
    elif current_var and '"' in line:
        value, desc = re.findall(r'(\d+)\s+"([^"]+)"', line)[0]
        if current_var in variable_info:
            variable_info[current_var]['values'][int(value)] = desc

### VARIABLE COLUMN SPACINGS IN DAT file

In [ ]:
dct_file = r'path/to/.DCT/file'
column_info = []

with open(dct_file, 'r') as file:
    lines = file.readlines()

for line in lines[2:-1]:
    parts = line.split()
    if len(parts) == 4:
        var_type = parts[0]
        subname = parts[1]
        start_pos = int(parts[3].split('-')[0]) - 1
        end_pos = int(parts[3].split('-')[1])
        column_info.append((subname, var_type, start_pos, end_pos))
    elif len(parts) == 3:
        var_type = parts[0]
        subname = parts[1]
        start_pos = int(parts[2].split(':')[1].split('-')[0]) - 1
        end_pos = int(parts[2].split(':')[1].split('-')[1])
        column_info.append((subname, var_type, start_pos, end_pos))
    else:
        print("ERROR: Invalid line:", line)

### READ GEO FILE TO GET COLUMNS TO READ

In [ ]:
import geopandas as gpd

# Path to the shapefile
shapefile_path = r"path/to/.SHP/file"

# Load the shapefile
gdf = gpd.read_file(shapefile_path)


# Extract only the necessary columns from the GPS data (cluster number and coordinates)
gdf = gdf[['DHSCLUST', 'LATNUM', 'LONGNUM']]

# Define new bounds for bombay
top_left = {'lat': 19.269802785051485, 'long': 72.7749741438024}
bottom_right = {'lat': 18.894288353461903, 'long': 72.9847334143265}

# Filter the geodataframe for clusters within these bounds
gdf = gdf[
    (gdf['LATNUM'] <= top_left['lat']) & 
    (gdf['LATNUM'] >= bottom_right['lat']) & 
    (gdf['LONGNUM'] >= top_left['long']) & 
    (gdf['LONGNUM'] <= bottom_right['long'])
]

# Get the DHSCLUST values within the bounds
dhsclust_within_bounds = gdf['DHSCLUST'].tolist()
print("DHSCLUST values within bounds:", dhsclust_within_bounds)

In [ ]:
gdf.head()

### READ DAT FILE FOR VALUES

In [ ]:
data_file = 'path/to/.DAT/file'
data = []

with open(data_file, 'r') as file:
    for line in file:
        record = {}
        cluster_value = int(line[column_info[2][2]:column_info[2][3]].strip())  # v001 is the third variable
        if cluster_value in dhsclust_within_bounds:
            for subname, var_type, start_pos, end_pos in column_info:
                value = line[start_pos:end_pos].strip()
                try:
                    if var_type in ['int', 'byte', 'long']:
                        value = int(value)
                    elif var_type == 'float':
                        value = float(value)
                except ValueError:
                    value = pd.NA
                record[subname] = value
            data.append(record)

### CREATE DATAFRAME

In [ ]:
# Create DataFrame from data
df = pd.DataFrame(data)

averaged_data = {}

for var_name, info in variable_info.items():
    subnames = info['subnames']
    if subnames:
        # If only one subname exists
        if len(subnames) == 1:
            subname = subnames[0]
            # Check if the data type of the subname is numeric
            if pd.api.types.is_numeric_dtype(df[subname]):
                averaged_data[var_name] = df[subname]
        else:
            # Check if any of the subname columns are numeric
            if not df[subnames].select_dtypes(include=['number']).empty:
                # Average the subname values, skipping NaNs
                averaged_data[var_name] = df[subnames].mean(axis=1, skipna=True)

# Create the final DataFrame
averaged_df = pd.DataFrame(averaged_data)

# Print the DataFrame
print(averaged_df.head())

In [ ]:
# Merge the DataFrames
merged_df = pd.merge(gdf, averaged_df, how='inner', left_on='DHSCLUST', right_on='HV001')

# Print the merged DataFrame
print(merged_df.head())

In [ ]:
# Group by DHSCLUST and calculate the mean for each numeric column
merged_df = merged_df.groupby('DHSCLUST').mean()
merged_df

### LINK THE DATAFRAME AND HEIGHT VALUES

In [ ]:
pixel_radius = 2000//5

In [ ]:
import rasterio
import geopandas as gpd
from pyproj import Transformer
import numpy as np
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

# Function to compute average intensity within a radius
def average_intensity(image_data, row, col, pixel_radius):
    # Calculate the bounds of the window around the specified center (row, col)
    start_row = max(row - pixel_radius, 0)
    end_row = min(row + pixel_radius + 1, image_data.shape[0])
    start_col = max(col - pixel_radius, 0)
    end_col = min(col + pixel_radius + 1, image_data.shape[1])

    # Extract the subarray from the image data
    subarray = image_data[start_row:end_row, start_col:end_col]
    
    # Compute the mean
    return np.mean(subarray)

# Path to the raster file (make sure to use the correct path)
raster_path = '/path/to/target/image'

# Load the raster file using rasterio
with rasterio.open(raster_path) as src:
    affine = src.transform  # Get the affine transformation of the raster
    image_data = src.read(1)  # Read the first band of the raster

# Confirm that the data is loaded by checking the shape
print("Raster dimensions:", image_data.shape)

# Setup Transformer to convert from WGS84 to UTM43N
transformer = Transformer.from_crs("EPSG:4326", "EPSG:32643", always_xy=True)

# Assuming you have a GeoDataFrame `gdf` with columns 'LATNUM' and 'LONGNUM' for coordinates
# Prepare to add building heights to the GeoDataFrame
heights = []

for index, row in merged_df.iterrows():
    # Convert geographic (longitude, latitude) to UTM coordinates
    utm_x, utm_y = transformer.transform(row['LONGNUM'], row['LATNUM'])
    
    # Convert UTM coordinates to raster indices
    col, row_idx = ~affine * (utm_x, utm_y)
    col, row_idx = int(col), int(row_idx)
    
    # Check if the raster indices are within the bounds of the raster image
    if 0 <= row_idx < image_data.shape[0] and 0 <= col < image_data.shape[1]:
        # Append building height at the raster index to the list
        heights.append(average_intensity(image_data, row_idx, col, pixel_radius))
    else:
        # Append NaN if indices are out of bounds
        heights.append(np.nan)
        # Print relevant parameters for debugging
        print(f"Out of bounds: LATNUM={row['LATNUM']}, LONGNUM={row['LONGNUM']}, UTM_X={utm_x}, UTM_Y={utm_y}, Col={col}, Row={row_idx}")

# Assign the list of building heights to the GeoDataFrame
merged_df['building_height'] = heights
# Drop rows where building height is NaN
merged_df = merged_df.dropna(subset=['building_height'])
# Display the updated GeoDataFrame
print(merged_df.head())

### PLOT SCATTER PLOTS FOR VARIABLES

In [ ]:
# Define the variables to plot against building_height
variables = ['define', 'your', 'variables of interest']
colors = ['blue', 'green','orange', 'purple']  # Colors for each plot

# Loop through variables for plotting and correlation
for i, var in enumerate(variables):
    correlation = merged_df['building_height'].corr(merged_df[var])
    plt.figure(figsize=(6, 4))
    plt.scatter(merged_df['building_height'], merged_df[var], color=colors[i])
    plt.title(f'Building Height vs {var} - Correlation: {correlation:.2f}')
    plt.xlabel('Building Height')
    plt.ylabel(var)
    plt.show()

### PLOT BAR CHARTS BETWEEN CLUSTER VALUE AND CLUSTER NUMBERS

In [ ]:
import rasterio
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyproj import Transformer
from tqdm import tqdm

# Step 1: Load the clustered raster data
clustered_raster_path = '/path/to/clustered/image'
with rasterio.open(clustered_raster_path) as src:
    clustered_image = src.read(1)  # Read the first band
    transform = src.transform

# Step 2: Convert UTM43N coordinates to pixel coordinates
def utm43n_to_pixel(lon, lat, transform):
    transformer = Transformer.from_crs("epsg:4326", "epsg:32643", always_xy=True)
    utm_x, utm_y = transformer.transform(lon, lat)
    col, row = ~transform * (utm_x, utm_y)
    return int(row), int(col)

# Step 3: Assign each household to a cluster
clusters = []
for idx, row in tqdm(merged_df.iterrows(), total=merged_df.shape[0]):
    lat, lon, height = row['LATNUM'], row['LONGNUM'], row['building_height']
    pixel_row, pixel_col = utm43n_to_pixel(lon, lat, transform)
    try:
        cluster_value = clustered_image[pixel_row, pixel_col]
        clusters.append(cluster_value)
    except IndexError:
        clusters.append(-1)  # If the household is out of the raster bounds

merged_df.loc[:, 'cluster'] = clusters

# Filter out households that are out of bounds and exclude cluster -1 and 255
filtered_df = merged_df[(merged_df['cluster'] != -1) & (merged_df['cluster'] != 255)]

# Define the variables list
variables = ['define', 'your', 'variables of interest']

# Step 4: Plot each variable against clusters
for variable in variables:
    # Group by cluster and get the value counts for each variable
    grouped_counts = filtered_df.groupby('cluster')[variable].value_counts().unstack(fill_value=0)

    # Calculate average values for each cluster
    average_values = filtered_df.groupby('cluster')[variable].mean()

    # Create a bar plot
    ax = grouped_counts.plot(kind='bar', stacked=True, figsize=(12, 8))

    ax.set_title(f'Distribution of {variable} by Cluster')
    ax.set_xlabel('Cluster')
    ax.set_ylabel('Count')
    
    # Add average values to the legend
    avg_labels = [f'Avg for Cluster {i}: {avg:.2f}' for i, avg in average_values.items()]
    ax.legend(title=variable, labels=avg_labels, loc='upper right')

    plt.show()
    plt.close()

### FILTER THE RELEVANT VARIABLES USING A LLM

In [ ]:
import requests
import time

# Hugging Face model endpoint
model_name = "gpt2"

# Hugging Face API URL
url = f"https://api-inference.huggingface.co/models/{model_name}"

# Obtain your Hugging Face Bearer token from environment variables or elsewhere
token = 'your token'

# List of non-relevant terms
non_relevant_terms = [
    "cluster number", "district", "aadhaar", "phone number", "ID", "identification", 
    "line number", "address", "record number", "identifier", "supervisor",
    "language", "error", "area unit", "day of", "date of", "month of", "year of"
]

def is_relevant(description):
    # Check if the description contains any non-relevant term
    if any(term in description.lower() for term in non_relevant_terms):
        return False

    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    payload = {
        "inputs": (
            f"Consider a variable description from the DHS VII survey dataset for India. "
            f"Determine if the variable is related to socio-economic or health indicators. "
            f"Examples of irrelevant variables include: cluster number, aadhaar number, phone number, state, district, district number and supervisor ID. "
            f"Examples of relevant variables include: education level, number of household members, wall/floor material, source of water, health measures like BMI, wealth measures, drainage facilities, and access to hospitals. "
            f"Now, given the following variable description, is it relevant to socio-economic or health indicators? Description: {description}. Respond with 'Yes' or 'No'."
        ),
        "parameters": {
            "max_new_tokens": 100,
            "do_sample": False,
            "return_full_text": False,
            "stop_sequence": "'"
        }
    }

    retries = 10  # Number of retries
    backoff_time = 10  # Initial backoff time in seconds

    for attempt in range(retries):
        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()  # Raise HTTPError for bad responses

            # Assuming response_json is a list and we want the first element
            response_json = response.json()
            if isinstance(response_json, list):
                generated_text = response_json[0]['generated_text']
            else:
                generated_text = response_json['generated_text']

            return 'Yes' in generated_text

        except requests.HTTPError as e:
            if response.status_code == 429:
                # Handle rate limiting
                print(f"Rate limited. Retrying in {backoff_time} seconds.")
                time.sleep(backoff_time)
                backoff_time *= 2  # Exponential backoff
            else:
                # For other HTTP errors, try again after a short delay
                print(f"HTTP Error occurred: {e}. Retrying...")
                time.sleep(2)
        except (KeyError, IndexError, requests.RequestException) as e:
            # For other errors, try again after a short delay
            print(f"Error occurred: {e}. Retrying...")
            time.sleep(2)

    # If the maximum retries are reached, log and return False
    print("Reached maximum retries. Please try again later.")
    return False

### CREATE CORRELATION TABLE FOR RELEVANT VARIABLES

In [ ]:
# Define a list to store correlations
correlation_list = []

# Iterate over each variable in variable_info
for var_name, info in variable_info.items():
    # Check if the variable exists as a column in merged_df
    if var_name in merged_df.columns:
        if is_relevant(info['description']) :
            # Calculate the correlation between the variable and building_height
            correlation = merged_df[var_name].corr(merged_df['building_height'])
            # Store the correlation value along with the variable name and description
            correlation_list.append((var_name, info['description'], info['values'], correlation))
        else:
            print("NOT RELEVANT", var_name, info['description'])

# Create a DataFrame from the correlation list
correlation_df = pd.DataFrame(correlation_list, columns=['Variable Name', 'Description', 'Range of Values', 'Correlation'])

# Sort the correlations in descending order based on their absolute values
correlation_df['Absolute Correlation'] = correlation_df['Correlation'].abs()
correlation_df = correlation_df.sort_values(by='Absolute Correlation', ascending=False).drop(columns='Absolute Correlation')

### TABULATE THE FINAL CORRELATION TABLE

In [ ]:
import pandas as pd
from tabulate import tabulate


# Convert correlation DataFrame to tabular format
correlation_table = correlation_df[['Variable Name', 'Description', 'Range of Values', 'Correlation']].values.tolist()

# Format the table using tabulate
formatted_table = tabulate(correlation_table, headers=['Variable Name', 'Description', 'Range of Values', 'Correlation'], tablefmt='grid')

# Print the table (optional)
print(formatted_table)

# Save the table to a text file
with open('MUMBAI_IR_RANKED_CORRELARIONS.txt', 'w') as f:
    f.write(formatted_table)